In [1]:
from fastai.vision.all import *

# Dataset preparations

In [2]:
def setup_mnist():
    save_path = Path('/storage/data/mnist_tensor') 
    
    target_files = ['training_x.save', 'training_y.save', 'testing_x.save', 'testing_y.save']
    loaded = []

    all_valid = np.all([(save_path/target_file).is_file() for target_file in target_files])
    if not all_valid:
        print('Setup from scratch')
        path = untar_data(URLs.MNIST)
        
        for mode in ['training', 'testing']:
            print(mode)
            x = []
            y = []

            for sub_folder in (path/mode).ls():
                category = sub_folder.name
                print(category)
                for img in sub_folder.glob('*.png'):
                    x.append(tensor(Image.open(img)))
                    y.append(int(category))
                    
            x = torch.stack(x).view(-1, 28*28).float()/255.
            y = tensor(y)
            
            torch.save(x, save_path/(mode+'_x.save'))
            torch.save(y, save_path/(mode+'_y.save'))
        
    
    return [torch.load(save_path/target_file) for target_file in target_files]    

In [3]:
train_x, train_y, test_x, test_y = setup_mnist()
train_x.shape

torch.Size([60000, 784])

# Dataloader

In [4]:
def ds_from_tensors(x,y):
    return list(zip(train_x, train_y))

In [5]:
train_ds = ds_from_tensors(train_x, train_y)
valid_ds = ds_from_tensors(test_x, test_y)

dls = DataLoaders.from_dsets(train_ds, valid_ds)

# Learner

In [16]:
model = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

In [17]:
class Learner:
    def __init__(self, model, dls, lr=1e-1):
        self.model = model.to('cuda')
        self.dls = dls.to('cuda')
        self.opt = Adam(self.model.parameters(), lr=lr)
        
    def _calc_gradients(self, xb, yb):
        preds = self.model(xb)
        loss = F.cross_entropy(preds, yb)
        loss.backward()
        return loss.item()
        
    def fit(self, n_epochs):
        for epoch in range(n_epochs):
            batch_loss = 0.
            # train
            for xb, yb in self.dls.train:
                self.opt.zero_grad()
                batch_loss += self._calc_gradients(xb, yb)
                self.opt.step()
            
            # validate
            correct = 0
            for xb, yb in self.dls.valid:
                correct += (model(xb).argmax(dim=1)==yb).float().sum()
            accuracy = (correct / len(self.dls.valid_ds)).item()
            print(epoch, accuracy)

In [18]:
l = Learner(model, dls, lr=1e-2)
l.fit(20)

0 0.9512166976928711
1 0.9657666683197021
2 0.9750000238418579
3 0.9765333533287048
4 0.9757333397865295
5 0.9753167033195496
6 0.9814167022705078
7 0.9800500273704529
8 0.9827666878700256
9 0.9819666743278503
10 0.9799833297729492
11 0.984000027179718
12 0.9791833162307739
13 0.9789666533470154
14 0.9815333485603333
15 0.982450008392334
16 0.9861833453178406
17 0.9816666841506958
18 0.9869500398635864
19 0.9838833212852478
